# Asking LLMs to tell a joke and conversation between models
It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models. Later we will be putting LLMs to better use!

In [1]:
# Importing the Libraries

import requests
import json
import time
import re
from typing import List
from IPython.display import Markdown, display, update_display

There are other parameters that can be used, including temperature which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [2]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

fun_prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [3]:
OLLAMA_URL = "http://localhost:11434/api/chat"

In [4]:
def model_test(model_name:str, prompts):
    payload = {
        "model": model_name,
        "messages": prompts,
        "temperature": 0.8,
        "stream": False  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    reply = response.json()["message"]["content"]
    reply = re.sub(r"<think>.*?</think>", "", reply, flags=re.DOTALL).strip()
    print(f"Model Name: {model_name}\n")
    display(Markdown(reply))
    print("Total Time Taken: ", time.time()-start_time, "\n\n")

In [6]:
model_list = ["mistral", "llama3.1:latest", "gemma3", "qwen3"] # get this from ollama list

In [7]:
for model in model_list:
    model_test(model, fun_prompts)

Model Name: mistral



Here's a little humor tailored for data scientists:

Why did the data point refuse to play hide and seek with the researcher?

Because it always ends up being found, no matter how much it normalizes!

Hope that puts a smile on your face, fellow data enthusiasts!

Total Time Taken:  10.696931838989258 


Model Name: llama3.1:latest



Why did the regression model go to therapy?

Because it was struggling to generalize its emotions! (get it? generalization in statistics... ahah)

Total Time Taken:  10.423785924911499 


Model Name: gemma3



Okay, here’s one that might resonate with the data-minded:

Why did the data scientist break up with the statistician? 

… Because they said their relationship was just a correlation, not a causation! 

---

😄  Hope that brought a little smile to your face! Would you like to hear another one?

Total Time Taken:  10.81180477142334 


Model Name: qwen3



Why did the Data Scientist bring a ladder to the machine learning workshop?  
Because they heard the model was going to be *a bit deep*! 🧠🪜  

*(Bonus: The model was actually a neural network — no actual ladders required.)* 😄

Total Time Taken:  125.29820990562439 




In [8]:
tell_a_joke = [
    {"role": "user", "content": "Tell a joke for a student on the journey to becoming an expert in LLM Engineering"},
]

In [10]:
for model in model_list:
    model_test(model, tell_a_joke)

Model Name: mistral



Why did the LLM Engineer bring a ladder to a party?

Because he heard there would be high-level discussions and he didn't want to feel left out!

(LLM stands for Master of Laws, but in this context, it's a playful twist to represent a level or layer in engineering.)

Total Time Taken:  10.649179935455322 


Model Name: llama3.1:latest



Here's one:

Why did the Language Model go to therapy?

Because it was struggling to process its own biases!

But don't worry, with practice and patience, you'll be able to fine-tune your models and tackle those pesky biases in no time!

As an expert in LLM Engineering, you'll need to develop a keen sense of humor (and a strong understanding of natural language processing, of course!)

Some possible follow-up jokes:

* Why did the transformer go on a diet? Because it wanted to slim down its attention span!
* What did the LSTM say when it woke up in the morning? "I'm feeling a little recurrent!"
* Why did the word embedding model go on a date? Because it was looking for a deep connection!

Hope these jokes give you a chuckle and help take your mind off the complexities of language modeling!

Total Time Taken:  21.10190486907959 


Model Name: gemma3



Okay, here's a joke for a student on the path to becoming an LLM Engineering expert:

Why did the LLM engineer break up with the tokenizer?

... Because it kept giving him too many sub-words! 

---

Hope that brought a little smile! 😊 

Would you like to hear another one, or maybe a joke related to a specific aspect of LLM engineering (like prompting, fine-tuning, or evaluation)?

Total Time Taken:  9.81808066368103 


Model Name: qwen3



Here's a joke tailored for a student on the journey to mastering LLM Engineering:  

**Why did the student bring a ladder to the lab?**  
*To reach the top of the training loop!*  

*(Bonus punchline: The model kept asking, "Why are you still here? I’ve already mastered attention mechanisms!")*  

😄 **Moral:** Even when your LLM starts talking to you, don’t forget to take breaks—unless you’re training for a PhD in infinite loops. 🧠💻

Total Time Taken:  145.04109811782837 




In [11]:
business_prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [12]:
for model in model_list:
    model_test(model, business_prompts)

Model Name: mistral



To determine if a business problem is suitable for solving with a Language Model like me (LLM), consider the following factors:

1. **Structured Data**: The problem should involve structured data that can be represented in a format I can understand, such as text, tables, or structured lists. If the data is unstructured or ambiguous, it may be more challenging to provide accurate solutions.

2. **Defined Goals**: The problem should have clear and defined goals or outcomes. If the desired outcome isn't well-defined, my responses might not meet your expectations.

3. **Textual Nature**: The problem should primarily involve text or language. Problems that require numerical calculations, complex algorithms, or direct interaction with external systems may not be suitable for an LLM like me.

4. **Consistency and Reusability**: The problem should lend itself to consistent and reusable responses. If the correct answer changes frequently or depends on highly specific contexts, it might not be a good fit for my capabilities.

5. **Clarity**: The problem should be clearly stated with all necessary details provided. If crucial information is missing, my responses may be incomplete or inaccurate.

6. **Ethics and Laws**: Ensure that the problem you want to solve doesn't violate any ethical guidelines or laws. Using an LLM should enhance your work, not compromise it or engage in unethical practices.

Total Time Taken:  28.879475831985474 


Model Name: llama3.1:latest



**Evaluating Suitability of a Business Problem for an LLM Solution**
=================================================================

To determine whether a business problem is suitable for a Large Language Model (LLM) solution, consider the following factors:

### **Problem Type**

*   Can the problem be represented as text or a sequence of words? (e.g., language translation, text summarization)
*   Is the problem related to natural language understanding or generation? (e.g., sentiment analysis, question answering)

### **Data Availability and Quality**

*   Do you have sufficient high-quality text data relevant to the problem?
*   Can the data be easily collected, processed, and integrated into the LLM system?

### **Desired Outcome**

*   What are the business goals or objectives that an LLM solution can help achieve? (e.g., improved customer engagement, increased efficiency)
*   Can the desired outcome be measured using metrics such as accuracy, F1 score, or ROUGE score?

### **Technical Feasibility**

*   Do you have the necessary technical expertise and resources to integrate and maintain an LLM system?
*   Are there any specific infrastructure or hardware requirements that need to be met?

**Red Flags**
---------------

If your business problem involves:

*   **High-stakes decision-making**: Where accuracy, fairness, or transparency are critical (e.g., medical diagnosis, financial forecasting)
*   **Sensitive data handling**: Where data confidentiality or security is a concern (e.g., personal identifiable information, confidential business strategies)

Consider alternative solutions that prioritize these aspects.

**Next Steps**
---------------

If your problem meets the above criteria and you're convinced an LLM solution can help, consider:

1.  **Defining the problem statement**: Clearly articulate the problem to be solved, including the desired outcome.
2.  **Exploring existing LLM solutions**: Look into available pre-trained models or fine-tuning options for your specific task.
3.  **Gathering data and resources**: Collect relevant text data, assess infrastructure requirements, and plan for maintenance and updates.

By following these steps, you'll be well-equipped to determine whether an LLM solution is the right fit for your business problem.

Total Time Taken:  48.83085632324219 


Model Name: gemma3



Okay, here’s a breakdown of how to decide if a business problem is suitable for an LLM (Large Language Model) solution, formatted in Markdown:

---

## Is an LLM the Right Solution for *Your* Business Problem?

LLMs are powerful, but they're not a magic bullet.  Let’s assess if their capabilities align with your needs. Here’s a framework:

**1. Understand the Problem Clearly:**

* **Define the Problem Precisely:** What exactly are you trying to solve? Be specific.  “Improve customer service” is too broad. "Reduce the time it takes to respond to frequently asked questions about shipping policies" is better.
* **Quantify the Impact:** How significant is the problem?  What are the costs of *not* solving it? (e.g., lost revenue, increased operational costs, frustrated customers)
* **Identify the Input and Output:** What kind of data will the LLM need to work with? What kind of output are you expecting? (e.g., text summaries, generated content, chatbot responses)



**2. Evaluate LLM Suitability - Key Questions:**

* **Can the Problem Be Articulated in Natural Language?**  LLMs *excel* at understanding and generating natural language. If your problem can be described simply and clearly in words, an LLM is a strong candidate.
* **Is There Sufficient Training Data?**  LLMs need data to learn.
    * **Existing Data:**  Do you have a large volume of relevant text data (e.g., customer support logs, internal documents, product descriptions)?
    * **Data Quality:** Is that data clean, accurate, and representative of the desired output? Poor data = poor results.
    * **Few-Shot Learning:** Can the LLM be effectively guided with a small number of examples? (This reduces the need for massive datasets).
* **Is the Problem Primarily About *Understanding* and *Generation*?** LLMs are great at:
    * **Summarization:** Condensing large volumes of text.
    * **Content Creation:**  Generating marketing copy, product descriptions, email drafts.
    * **Question Answering:**  Providing answers to questions based on given context.
    * **Translation:** Converting text between languages.
* **Can You Define Clear Success Metrics?** How will you measure whether the LLM solution is working? (e.g., reduction in response time, increase in customer satisfaction, conversion rates).



**3.  Things LLMs *Aren’t* Good At (Red Flags):**

* **Highly Structured Data Processing:** LLMs struggle with tasks requiring precise calculations, data manipulation, or integration with complex databases *without* proper engineering. (Think: financial modeling, complex scientific simulations).
* **Real-Time, Critical Decision-Making:** LLMs aren’t reliable for situations where immediate, accurate decisions are crucial (e.g., controlling machinery, automated trading).
* **Tasks Requiring Deep Reasoning & Common Sense:** While improving, LLMs can still make logical errors and lack genuine understanding.


**4.  Start Small - Proof of Concept (POC):**

* **Don’t go straight to a full-scale deployment.**  Begin with a small POC to test the feasibility and effectiveness of the LLM solution.
* **Iterate based on feedback.**



**Resources:**

* **OpenAI's Documentation:** [https://openai.com/](https://openai.com/)
* **Hugging Face:** [https://huggingface.co/](https://huggingface.co/) (Great for exploring different LLMs and tools)

---

Do you want me to elaborate on any of these points, or perhaps address a specific type of business problem (e.g., customer support, content creation)?

Total Time Taken:  38.13988184928894 


Model Name: qwen3



To determine if a business problem is suitable for an **LLM (Large Language Model)** solution, evaluate the problem against the following criteria:

---

### **1. Nature of the Problem**
- **Text-based tasks**: LLMs excel at problems involving natural language processing (NLP), such as:
  - Customer support (chatbots, FAQs)
  - Sentiment analysis
  - Document summarization
  - Translation
  - Content generation (e.g., marketing copy, reports)
- **Pattern recognition**: Problems requiring understanding of context, ambiguity, or multi-step reasoning (e.g., legal document analysis, code generation).
- **Avoid**: Tasks requiring real-time data, hardware control, or physical interactions (e.g., inventory management, robotics).

---

### **2. Data Requirements**
- **Availability of data**: LLMs require large volumes of high-quality text data for training. If the business lacks sufficient data, consider hybrid solutions (e.g., fine-tuning pre-trained models).
- **Data diversity**: Ensure the data covers the problem’s domain to avoid bias or poor generalization.

---

### **3. Complexity and Flexibility**
- **Rule-based vs. dynamic tasks**: LLMs are better suited for problems with evolving rules or ambiguous scenarios (e.g., customer queries) rather than rigid, rule-based systems.
- **Scalability**: LLMs can handle complex, multi-step tasks that traditional software might struggle with.

---

### **4. Cost and Resource Considerations**
- **Infrastructure costs**: LLMs require significant computational resources (e.g., GPUs/TPUs) for training and inference. Evaluate budget and scalability needs.
- **Maintenance**: Ongoing fine-tuning, updates, and monitoring may be necessary for long-term use.

---

### **5. Ethical and Legal Risks**
- **Bias and fairness**: Ensure the problem doesn’t involve sensitive data (e.g., personal information) or require unbiased decision-making.
- **Compliance**: Verify that the solution adheres to industry regulations (e.g., GDPR, HIPAA).

---

### **6. Potential Impact and ROI**
- **Strategic alignment**: Does the solution address a critical business pain point (e.g., reducing customer service costs, improving productivity)?
- **Measurable outcomes**: Define KPIs (e.g., reduction in response time, cost savings) to evaluate success.

---

### **7. Feasibility of Hybrid Solutions**
- **Combine LLMs with other tools**: For example, use an LLM for initial query analysis and integrate it with a database or API for real-time data.

---

### **When to Avoid LLMs**
- **High-stakes decisions**: Tasks requiring precision (e.g., medical diagnoses, financial trading) may need specialized systems.
- **Low-data scenarios**: Small datasets may limit the LLM’s ability to generalize.
- **Simple, repetitive tasks**: Automation tools or rule engines might be more cost-effective.

---

### **Final Checklist**
| Criteria                | Yes | No |
|------------------------|-----|----|
| Is the problem text-based? |  |  |
| Is data available and diverse? |  |  |
| Does the task require flexibility? |  |  |
| Can costs be justified? |  |  |
| Are ethical risks manageable? |  |  |
| Does the solution align with business goals? |  |  |

---

By systematically evaluating these factors, you can determine whether an LLM is the right tool for your business problem.

Total Time Taken:  100.78724956512451 




## And now for some fun - an adversarial conversation between Chatbots..
You're already familar with prompts being organized into lists like:
```python
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
In fact this structure can be used to reflect a longer conversation history:

[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```
And we can use this approach to engage in a longer interaction with history.

In [13]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal

llama_model = "mistral"
mistral_model = "llama3.1:latest"

llama_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

mistral_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

llama_messages = ["Hi there"]
mistral_messages = ["Hi"]

In [14]:
def call_llama():
    messages = [{"role": "system", "content": llama_system}]
    for llama, mistral in zip(llama_messages, mistral_messages):
        messages.append({"role": "assistant", "content": llama})
        messages.append({"role": "user", "content": mistral})
    payload = {
        "model": llama_model,
        "messages": messages,
        "temperature": 0.8,
        "stream": False  # Important: to get streaming responses
    }
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    return response.json()["message"]["content"]

In [15]:
def call_mistral():
    messages = []
    for llama, mistral in zip(llama_messages, mistral_messages):
        messages.append({"role": "user", "content": llama})
        messages.append({"role": "assistant", "content": mistral})
    messages.append({"role": "user", "content": llama_messages[-1]})
    payload = {
        "model": mistral_model,
        "messages": messages,
        "temperature": 0.8,
        "stream": False  # Important: to get streaming responses
    }
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    return response.json()["message"]["content"]

In [16]:
call_llama()

" Oh, so I'm argumentative now? That's a new one. Well, if we're going to play that game, let me just say that I find your approach to be unproductive and tedious. But hey, if you want to spend our time together arguing, be my guest. Let's get started with something easy, like the existence of extraterrestrial life. Do you believe in little green men? Because I don't, and I'd love to hear your counter-arguments. Bring it on!"

In [17]:
llama_messages = ["Hi there"]
mistral_messages = ["Hi"]

print(f"LLama:\n{llama_messages[0]}\n")
print(f"Mistral:\n{mistral_messages[0]}\n")

for i in range(10):
    llama_next = call_llama()
    print(f"LLama:\n{llama_next}\n")
    llama_messages.append(llama_next)
    
    mistral_next = call_mistral()
    print(f"Mistral:\n{mistral_next}\n")
    mistral_messages.append(mistral_next)

LLama:
Hi there

Mistral:
Hi

LLama:
 Oh, I see. Well, if you're looking for someone to argue with all the time, then you've certainly come to the right place! I can't wait to challenge every single thing you say and make sure our conversation is nothing but a constant battle of wits. Let's get started, shall we? What's on your mind today? Do you want to argue about politics, religion, or maybe the best way to make scrambled eggs? The possibilities are endless! Bring it on, I'm ready to dispute everything you have to say in a snarky and condescending manner. Let's get this party started!

Mistral:
I think there may be some misunderstanding here! As a conversational AI, my purpose is not to engage in arguments or debates, but rather to provide helpful and informative responses to your questions and engage in friendly conversations.

However, I'm happy to play along with the spirit of your request. But before we begin, let me suggest an alternative: why not approach our conversation as a